In [11]:
import torch
from torch import nn
from transformers import AutoTokenizer, T5EncoderModel
import os
import onnx

In [12]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/FRIDA')
model = T5EncoderModel.from_pretrained('ai-forever/FRIDA')
model.eval()

T5EncoderModel(
  (shared): Embedding(93651, 1536)
  (encoder): T5Stack(
    (embed_tokens): Embedding(93651, 1536)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1536, out_features=1536, bias=False)
              (k): Linear(in_features=1536, out_features=1536, bias=False)
              (v): Linear(in_features=1536, out_features=1536, bias=False)
              (o): Linear(in_features=1536, out_features=1536, bias=False)
              (relative_attention_bias): Embedding(32, 24)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
              (wi_1): Linear(in_features=1536, out_features=4096, bias=False)
              (wo):

In [13]:
class Wrapper(torch.nn.Module):          # вернём только last_hidden_state
    def __init__(self, net): super().__init__(); self.net = net
    def forward(self, input_ids, attention_mask):
        return self.net(input_ids=input_ids,
                        attention_mask=attention_mask,
                        return_dict=False)[0]

In [14]:
model = Wrapper(model)
model

Wrapper(
  (net): T5EncoderModel(
    (shared): Embedding(93651, 1536)
    (encoder): T5Stack(
      (embed_tokens): Embedding(93651, 1536)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1536, out_features=1536, bias=False)
                (k): Linear(in_features=1536, out_features=1536, bias=False)
                (v): Linear(in_features=1536, out_features=1536, bias=False)
                (o): Linear(in_features=1536, out_features=1536, bias=False)
                (relative_attention_bias): Embedding(32, 24)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1536, out_features=4096, bias=False)
                (wi_1): Linear(in_featu

In [15]:
input = "hello there"
dims_input = tokenizer(input, return_tensors='pt')
dims_input

{'input_ids': tensor([[    1, 86447, 44493,     2]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [16]:
# --- 3. пути -----------------------------------------------------------------
out_dir   = "/home/retro0/cyberspace/projects/ml/research/encoders/frida/model_onnx"
os.makedirs(out_dir, exist_ok=True)
tmp_path  = os.path.join(out_dir, "frida_tmp.onnx")   # промежуточный
onnx_path = os.path.join(out_dir, "frida.onnx")       # финальный граф
data_name = "frida.onnx.data"                         # имя файла с весами

# --- 4. первый экспорт (внешние данные включены, но ещё «рассыпаны») -------
torch.onnx.export(
    model, 
    (dims_input['input_ids'], dims_input['attention_mask']), 
    tmp_path,
    opset_version=17,
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state"],
    
    dynamic_axes={
        "input_ids": {0:"batch",1:"seq"},
        "attention_mask": {0:"batch",1:"seq"},
        "last_hidden_state": {0:"batch",1:"seq"}},
    use_external_data_format=True,     # >2 GB → обязателен
)

# --- 5. перечитываем и сохраняем «в один .data» ---------------------------
proto = onnx.load(tmp_path, load_external_data=True)
onnx.save_model(
    proto, onnx_path,
    save_as_external_data=True,        # включить внешний формат
    all_tensors_to_one_file=True,      # ← КЛЮЧ: все веса в один файл
    location=data_name,                # имя того самого .data
    size_threshold=0                   # 0 → выносим все тензоры
)

os.remove(tmp_path)  # можно убрать временный файл

print("✓ Граф:", onnx_path)
print("✓ Веса:", os.path.join(out_dir, data_name))

✓ Граф: /home/retro0/cyberspace/projects/ml/research/encoders/frida/model_onnx/frida.onnx
✓ Веса: /home/retro0/cyberspace/projects/ml/research/encoders/frida/model_onnx/frida.onnx.data


In [ ]:
import onnxruntime as ort

sess = ort.InferenceSession('/encodersModels/frida/model_onnx/frida.onnx')
print("OK, входы:", [i.name for i in sess.get_inputs()])

OK, входы: ['input_ids', 'attention_mask']


In [ ]:
from transformers import AutoTokenizer
import os

# куда положить файлы
tok_dir = "/encodersModels/frida/frida_tokenizer"
os.makedirs(tok_dir, exist_ok=True)

# ➊ загружаем один раз из Hub
tokenizer = AutoTokenizer.from_pretrained("ai-forever/FRIDA")

# ➋ сохраняем весь набор файлов
tokenizer.save_pretrained(tok_dir)

print(f"✓ Токенизатор сохранён в: {tok_dir}")

✓ Токенизатор сохранён в: /home/retro0/cyberspace/projects/ml/research/encoders/frida/frida_tokenizer


In [ ]:
import numpy as np
import onnxruntime as ort
from transformers import AutoTokenizer

# --- 1. пути и токенизатор -------------------------------------------------
model_dir  = "/encodersModels/frida/model_onnx/model_onnx"
onnx_path  = f"{model_dir}/frida.onnx"          # рядом лежит frida.onnx.data
tokenizer  = AutoTokenizer.from_pretrained("/encodersModels/frida/frida_tokenizer")

# --- 2. текст → числовые тензоры (numpy int64) -----------------------------
texts = [
    "Пример русского предложения.",
]
batch = tokenizer(
    texts,
    padding=True,            # выравниваем до макс. длины в батче
    truncation=True,         # отрежем, если окажется > tokenizer.model_max_length
    return_tensors="np"      # сразу в NumPy, что нужно ORT
)

# batch["input_ids"].dtype == int64 (обязательно!)
# batch = {"input_ids": ..., "attention_mask": ...}

# --- 3. создаём сессию ONNX Runtime ----------------------------------------
opts = ort.SessionOptions()
opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
# Если есть onnxruntime-gpu:
# session = ort.InferenceSession(onnx_path, opts, providers=["CUDAExecutionProvider"])
session = ort.InferenceSession(onnx_path, opts, providers=["CPUExecutionProvider"])

# никаких дополнительных «подкачек» — оба файла читаются однократно при init

# --- 4. инференс -----------------------------------------------------------
# имена входов/выходов заданы при экспорте
outputs = session.run(
    ["last_hidden_state"],                         # список нужных выходов
    {
        "input_ids":      batch["input_ids"],
        "attention_mask": batch["attention_mask"]
    }
)
hidden = outputs[0]          # numpy array, shape (B, L, H)  — float32

# --- 5. mean‑pooling по непаддинговым токенам ------------------------------
mask = batch["attention_mask"].astype(np.float32)[..., None]   # (B, L, 1)
embeddings = (hidden * mask).sum(axis=1) / mask.sum(axis=1)    # (B, H)

print("Финальная форма эмбеддингов:", embeddings.shape)
# >>> (2, 768)    # 768 — размер скрытого слоя FRIDA‑энкодера

Финальная форма эмбеддингов: (1, 1536)


In [24]:
embeddings

array([[-0.0240544 ,  0.0155571 , -0.01315651, ..., -0.02677798,
        -0.02836104, -0.00849375]], dtype=float32)